# Smartphones and Watches advisor
In this serie of notebooks we will:
- Load text from a Youtube video playlist
- Split loaded text into appropriate chunks
- Create a vector store to load chunk embeddings
- **Create a Question Answering chatbot**:

    - **with or without prompts engineering**
    - **with different chain types**
    - with or **without chat memory**

!['Data Connection'](../../images/les_numeriques_youtube/data_connection.png)


# Create a Question Answering chatbot

Create a Question Answering chatbot:
- With or without prompt engineering
- With three different chain types:
    - Map Reduce
    - Refine
    - Map rerank


!['Question Aswnering Process'](../../images/les_numeriques_youtube/question_answering.png)

In [1]:
# Imports 
# Env var
import os 
import sys
from dotenv import load_dotenv, find_dotenv

# Langchain
import openai
from langchain.llms import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# SelfQuery Retriever
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

# Compressor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# QA retriever
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# Prompt template
from langchain.prompts import PromptTemplate

In [2]:
# Env variable
sys.path.append('../..')
_ = load_dotenv(find_dotenv())

In [3]:
# Load vector db
persist_directory = 'docs/chroma/'

embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

print(vectordb._collection.count())

815


In [4]:
question = "Quelles sont les caractéristiques de l'iphone 14 ? "

In [5]:
docs = vectordb.similarity_search(question, k=5)
docs[:5]

[Document(page_content="Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.", metadata={'source': "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a", 'chunk': 0}),
 Document(page_content="Alors esthétiquement, il n'y a rien de nouveau par rapport aux iPhone 13 mini et aux iPhone 13 de l'année dernière. A l'intérieur, c'est quasiment la même chose, sauf que l'on retrouve la puce Apple A15 Bionic que l'on retrouvait dans les iPhone 13 Pro et 13 Pro Max en 2021. Du côté de l'iPhone 14, on garde le même gabarit qu'avec l'iPhone 13 et du côté de l'iPhone 14+, on prend le même gabarit qu'un iPhone 14 Pro Max.

### Without prompt engineering


In [6]:
llm = OpenAI(temperature=0)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
)

In [7]:
question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain({"query": question})
result["result"]

" L'iPhone 14 a un écran de 6,1 pouces, un gabarit similaire à l'iPhone 13, une puce Apple A15 Bionic, une batterie améliorée, des capteurs photo similaires à ceux de l'iPhone 13 Pro et un module frontal entièrement revu. Il n'a pas de nouveau design sur l'écran et n'a pas le droit au Dynamic Island."

### With prompt engineering

In [8]:
# Build prompt
template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 

Keep the answer as concise as possible. 
You can answer with bullet point as well.

Always say "Merci pour cette question!" at the end of the answer. 


CONTEXT: {context}
-------
Human: {question}
Assistant:

"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"], template=template)

In [9]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT} # Add a prompt
)

In [10]:
question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain({"query": question})

In [11]:
print(result["result"])

Les caractéristiques de l'iPhone 14 sont: 
- Écran de 6,1 pouces 
- Puce Apple A15 Bionic 
- Batterie améliorée 
- Capteurs photo du iPhone 13 Pro pour le principal et l'ultra grand-angle 
- Module frontal entièrement revu 
- Encoche 
- Pas de Dynamic Island 
Merci pour cette question!


## Question answering chain type
- Map Reduce
- Refine
- Map rerank

### Lang Chain platform

- Go to https://smith.langchain.com/
- Setting > Create API key > Copy and past it in your.env file 

In [12]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.langchain.plus"
# os.environ["LANGCHAIN_API_KEY"] = "[YOUR_API_KEY]"
# Go to https://smith.langchain.com/
# Setting > Create API key > Copy and past it in your.env file 

_ = load_dotenv(find_dotenv()) # read local .env file


### Map reduce

![''Map Reduce'](../../images/les_numeriques_youtube/map_reduce.png)


In [13]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain_mr({"query": question})
result["result"]

' The iPhone 14 features a 6.1 inch screen, the same design as the iPhone 13, improved battery, and the same photo sensors as the iPhone 13 Pro for the main and ultra-wide angle. The front module has been completely redesigned. The iPhone 14+ features a 6.7 inch screen, the same design as the iPhone 13, improved battery, and the same photo sensors as the iPhone 13 Pro for the main and ultra-wide angle. The front module has been completely redesigned.'

- It is a lot slower
- The results are worst

Why ?
- The answers are first based on each document independantly 
- Dependent chunks are not joined before final answer, only intermediate answers


## Refine method


!['Refine'](../../images/les_numeriques_youtube/refine.png)


In [14]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)

question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain_mr({"query": question})
print(result["result"])



L'iPhone 14 est doté d'un écran LCD de 6,1 pouces, d'un processeur A15 Bionic, d'un double appareil photo arrière de 12 mégapixels et d'une batterie de 2 815 mAh. Il est disponible en 64, 128 et 256 Go de stockage et est disponible en six couleurs différentes. Il a le même gabarit que l'iPhone 13 et l'iPhone 14+ a le même gabarit qu'un iPhone 14 Pro Max. De plus, l'iPhone 14+ est doté d'un écran de 6,7 pouces, et la batterie a été améliorée grâce à la nouvelle puce. En photo, les capteurs qui étaient présents sur les iPhone 13 Pro sont présents sur le principal et l'ultra grand-angle. Le module frontal a également été entièrement revu. Contrairement aux 14 Pro et 14 Pro Max, l'iPhone 14 n'a pas de n


## Rerank method


!['Map Rerank'](../../images/les_numeriques_youtube/map_rerank.png)


In [15]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_rerank"
)

question = "Quelles sont les caractéristiques de l'iPhone 14 ?"
result = qa_chain_mr({"query": question})
print(result["result"])

/Users/florian.bastin/Documents/Code/Octo/GenAI/langchain_discovery/.venv/lib/python3.10/site-packages/langchain/chains/llm.py:303: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


 L'iPhone 14 est l'entrée de gamme de la gamme d'iPhone 14. Il est doté d'un écran LCD de 6,1 pouces, d'un processeur A14 Bionic, d'un double capteur photo arrière de 12 mégapixels et d'une batterie de 2 815 mAh.


### RetrievalQA limitations
 
QA fails to preserve conversational history.

In [16]:
question = 'Quel iPhone viens-tu de me présenter cher LLM ?'

result = qa_chain_mr({"query": question})
print(result["result"])

 Tu as présenté un iPhone 13 Product Red, un iPhone 13 bleu et un iPhone 13 Pro.
